# Problem 1

In [5]:
import torch 
import torch.nn as nn

import numpy as np
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt

In [7]:
# NOTE ==============================================
#
# Fill in code for every method which has a TODO
#
# Your implementation should use the contract (inputs
# and outputs) given for each model, because that is 
# what the main script expects. If you modify the contract, 
# you must justify that choice, note it in your report, and notify the TAs 
# so that we run the correct code.
#
# You may modify the internals of the RNN and GRU classes
# as much as you like, except you must keep the methods
# in each (init_weights_uniform, init_hidden, and forward)
# Using nn.Module and "forward" tells torch which 
# parameters are involved in the forward pass, so that it
# can correctly (automatically) set up the backward pass.
#
# You should not modify the interals of the Transformer
# except where indicated to implement the multi-head
# attention. 


def clones(module, N):
    "A helper function for producing N identical layers (each with their own parameters)."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Problem 1
class RNN(nn.Module): # Implement a stacked vanilla RNN with Tanh nonlinearities.
  def __init__(self, emb_size, hidden_size, seq_len, batch_size, vocab_size, num_layers, dp_keep_prob):
    # TODO ========================
    # Initialization of the parameters of the recurrent and fc layers. 
    # Your implementation should support any number of stacked hidden layers 
    # (specified by num_layers), use an input embedding layer, and include fully
    # connected layers with dropout after each recurrent layer.
    # Note: you may use pytorch's nn.Linear, nn.Dropout, and nn.Embedding 
    # modules, but not recurrent modules.
    #
    # To create a variable number of parameter tensors and/or nn.Modules 
    # (for the stacked hidden layer), you may need to use nn.ModuleList or the 
    # provided clones function (as opposed to a regular python list), in order 
    # for Pytorch to recognize these parameters as belonging to this nn.Module 
    # and compute their gradients automatically. You're not obligated to use the
    # provided clones function.
    """
    emb_size:     The numvwe of units in the input embeddings
    hidden_size:  The number of hidden units per layer
    seq_len:      The length of the input sequences
    vocab_size:   The number of tokens in the vocabulary (10,000 for Penn TreeBank)
    num_layers:   The depth of the stack (i.e. the number of hidden layers at 
                  each time-step)
    dp_keep_prob: The probability of *not* dropping out units in the 
                  non-recurrent connections.
                  Do not apply dropout on recurrent connections.
    """
    super(RNN, self).__init__()
    
    # Parameters
    self.emb_size = emb_size
    self.hidden_size = hidden_size
    self.seq_len = seq_len
    self.batch_size = batch_size
    self.vocab_size = vocab_size
    self.num_layers = num_layers
    self.dp_keep_prob = dp_keep_prob
    
#     # RNN Parameters :
#     W_x = nn.ParameterList([nn.Parameter(torch.Tensor(hidden_size, emb_size))])
#     W_x.extend([nn.Parameter(torch.Tensor(hidden_size, hidden_size)) for _ in range(num_layers-1)])
#     W_h = nn.ParameterList([nn.Parameter(torch.Tensor(hidden_size, hidden_size)) for _ in range(num_layers)])
#     W_y = nn.Parameter(torch.Tensor(vocab_size, hidden_size))
    
#     b_h = nn.ParameterList([nn.Parameter(torch.Tensor(hidden_size)) for _ in range(num_layers)])
#     b_y = nn.Parameter(torch.Tensor(vocab_size))
    
#     self.weights = [W_x, W_h, W_y]
#     self.bias = [b_h, b_y]

    # RNN Layer : 
    first_rnn = nn.Linear(self.emb_size + self.hidden_size, self.hidden_size)
    self.recurrent_layers = nn.ModuleList([first_rnn])
    other_rnn = nn.Linear(2*self.hidden_size, self.hidden_size)
    self.recurrent_layers.extend(clones(other_rnn, self.num_layers-1))
        
    # Word Embedding Layer : 
    self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = emb_size)
    
    # FC Layers and Dropout Layers :
    self.linear_layers = clones(nn.Linear(hidden_size, hidden_size), num_layers-1)
    self.linear_layers.append(nn.Linear(hidden_size, vocab_size))
    
    self.dropout_layers = clones(nn.Dropout(p = 1-dp_keep_prob), num_layers)
    
    # Hidden layers coefficient: 
    #self.hidden_layers = torch.Tensor(self.num_layers, self.batch_size, self.hidden_size)

    

  def init_weights_uniform(self):
    # TODO ========================
    # Initialize all the weights uniformly in the range [-0.1, 0.1]
    # and all the biases to 0 (in place)
    for name, param in self.recurrent_layers.named_parameters():
        if name == 'weight':
            nn.init.uniform(param, -0.1, 0.1)
        else:
            nn.init.constant(param, 0)
    
    for name, param in self.linear_layers.named_parameters():
        if name == 'weight':
            nn.init.uniform(param, -0.1, 0.1)
        else:
            nn.init.constant(param, 0)
#     for i in range(self.num_layers):
#         nn.init.uniform_(self.weights[0][i], -0.1, 0.1)
#         nn.init.uniform_(self.weights[1][i], -0.1, 0.1)
#         nn.init.constant_(self.bias[0][i], 0)
        
#     nn.init.uniform_(self.weights[2], -0.1, 0.1)
#     nn.init.constant_(self.bias[1], 0)
    
    

  def init_hidden(self):
    # TODO ========================
    # initialize the hidden states to zero
    """
    This is used for the first mini-batch in an epoch, only.
    """
    #nn.init.constant_(self.hidden_layers, 0)
    return nn.init.constant_(torch.Tensor(self.num_layers, self.batch_size, self.hidden_size), 0)
    # a parameter tensor of shape (self.num_layers, self.batch_size, self.hidden_size)

  def forward(self, inputs, hidden):
    # TODO ========================
    # Compute the forward pass, using a nested python for loops.
    # The outer for loop should iterate over timesteps, and the 
    # inner for loop should iterate over hidden layers of the stack. 
    # 
    # Within these for loops, use the parameter tensors and/or nn.modules you 
    # created in __init__ to compute the recurrent updates according to the 
    # equations provided in the .tex of the assignment.
    #
    # Note that those equations are for a single hidden-layer RNN, not a stacked
    # RNN. For a stacked RNN, the hidden states of the l-th layer are used as 
    # inputs to to the {l+1}-st layer (taking the place of the input sequence).

    """
    Arguments:
        - inputs: A mini-batch of input sequences, composed of integers that 
                    represent the index of the current token(s) in the vocabulary.
                        shape: (seq_len, batch_size)
        - hidden: The initial hidden states for every layer of the stacked RNN.
                        shape: (num_layers, batch_size, hidden_size)
    
    Returns:
        - Logits for the softmax over output tokens at every time-step.
              **Do NOT apply softmax to the outputs!**
              Pytorch's CrossEntropyLoss function (applied in ptb-lm.py) does 
              this computation implicitly.
                    shape: (seq_len, batch_size, vocab_size)
        - The final hidden states for every layer of the stacked RNN.
              These will be used as the initial hidden states for all the 
              mini-batches in an epoch, except for the first, where the return 
              value of self.init_hidden will be used.
              See the repackage_hiddens function in ptb-lm.py for more details, 
              if you are curious.
                    shape: (num_layers, batch_size, hidden_size)
    """
    if inputs.is_cuda:
            device = inputs.get_device()
        else:
            device = torch.device("cpu")
    
    hidden = hidden.to(device)
    
    emb_inputs = self.embedding(inputs)
    logits = torch.Tensor(self.seq_len, self.batch_size, self.vocab_size).to(device)
    
    for seq in range(self.seq_len):
        
        hidden[0] = self.recurrent_layers[0](torch.cat([hidden[0],emb_inputs[seq]], 1))
        outputs = self.dropout_layers[0](hidden[0])
        outputs = self.linear_layers[0](outputs)
        if self.num_layers == 1:
            logits[seq] = outputs
        
        for l in range(1,self.num_layers):    
            hidden[l] = self.recurrent_layers[l](torch.cat([hidden[l],outputs], 1))
            outputs = self.dropout_layers[l](hidden[l])
            if l == self.num_layers-1:
                logits[seq] = self.linear_layers[l](outputs)
            else:
                outputs = self.linear_layers[l](outputs)
    return logits.view(self.seq_len, self.batch_size, self.vocab_size), hidden

  def generate(self, input, hidden, generated_seq_len):
    # TODO ========================
    # Compute the forward pass, as in the self.forward method (above).
    # You'll probably want to copy substantial portions of that code here.
    # 
    # We "seed" the generation by providing the first inputs.
    # Subsequent inputs are generated by sampling from the output distribution, 
    # as described in the tex (Problem 5.3)
    # Unlike for self.forward, you WILL need to apply the softmax activation 
    # function here in order to compute the parameters of the categorical 
    # distributions to be sampled from at each time-step.

    """
    Arguments:
        - input: A mini-batch of input tokens (NOT sequences!)
                        shape: (batch_size)
        - hidden: The initial hidden states for every layer of the stacked RNN.
                        shape: (num_layers, batch_size, hidden_size)
        - generated_seq_len: The length of the sequence to generate.
                       Note that this can be different than the length used 
                       for training (self.seq_len)
    Returns:
        - Sampled sequences of tokens
                    shape: (generated_seq_len, batch_size)
    """
    if inputs.is_cuda:
        device = inputs.get_device()
    else:
        device = torch.device("cpu")
    
    hidden = hidden.to(device)
    
    emb_inputs = self.embedding(inputs)
    logits = torch.Tensor(self.seq_len, self.batch_size, self.vocab_size).to(device)
    
    for seq in range(self.seq_len):
        
        hidden[0] = self.recurrent_layers[0](torch.cat([hidden[0],emb_inputs[seq]], 1))
        outputs = self.dropout_layers[0](hidden[0])
        outputs = self.linear_layers[0](outputs)
        if self.num_layers == 1:
            logits[seq] = outputs
        
        for l in range(1,self.num_layers):    
            hidden[l] = self.recurrent_layers[l](torch.cat([hidden[l],outputs], 1))
            outputs = self.dropout_layers[l](hidden[l])
            if l == self.num_layers-1:
                logits[seq] = self.linear_layers[l](outputs)
            else:
                outputs = self.linear_layers[l](outputs)
    return logits.view(self.seq_len, self.batch_size, self.vocab_size), hidden
    
    
    samples = torch.Tensor(generated_seq_len, batch_size)
    samples[0] = self.embedding(input) 
    for seq in range(generated_seq_len):
        for l in range(self.num_layers):
            hidden[l] = self.recurrent_layers[l](torch.cat([hidden[l],samples[seq]], 0))
            if l < self.num_layers-1:
                samples[seq] = self.dropout_layers[l](hidden[l])
            samples[seq] = self.linear_layers[l](samples[seq])
        if seq < generated_seq_len-1:
            samples[seq+1] = samples[seq]
    return samples

In [ ]:
runfile('C:/Users/Théo/Documents/PolyMtl/IFT6135_2019/02/ptb-lm.py', 
        wdir='C:/Users/Théo/Documents/PolyMtl/IFT6135_2019/02',
        args = "--model=RNN --optimizer=ADAM --initial_lr=0.0001 --batch_size=20 --seq_len=35 --hidden_size=1500 --num_layers=2 --dp_keep_prob=0.35 --save_best")

########## Running Main Loop ##########################

EPOCH 0 ------------------
step: 10        loss: 3529.5392274856567        speed (wps):902.8174428959136
step: 142       loss: 149279.704079628  speed (wps):905.5170478982327
step: 274       loss: 179141.8760561943 speed (wps):905.3147800750617
step: 406       loss: 208182.39816904068        speed (wps):905.3567834086997
step: 538       loss: 237026.87060117722        speed (wps):906.2892257028814
step: 670       loss: 265616.11352682114        speed (wps):882.6916074195929
step: 802       loss: 293532.5080084801 speed (wps):860.783327800122
step: 934       loss: 321377.4548268318 speed (wps):847.7529544398733
step: 1066      loss: 349061.966881752  speed (wps):838.0461397122924
step: 1198      loss: 376456.8313694    speed (wps):830.6268344897844
Saving model parameters to best_params.pt
epoch: 0        train ppl: 5858.483523467034    val ppl: 328.5533928651772      best val: 328.5533928651772     time (s) spent in epoch: 1144.3142385482788

EPOCH 1 ------------------
step: 10        loss: 2316.36301279068  speed (wps):766.8503092924894
step: 142       loss: 29103.100430965424        speed (wps):776.1474559565766
step: 274       loss: 56287.026228904724        speed (wps):775.6277608953311
step: 406       loss: 83102.58867263794 speed (wps):775.2052193173345
step: 538       loss: 110017.69404888153        speed (wps):773.8004343091643
step: 670       loss: 136931.1484694481 speed (wps):766.0637896028011
step: 802       loss: 163327.31778621674        speed (wps):760.6522099345661
step: 934       loss: 189825.20168066025        speed (wps):761.5092705630851
step: 1066      loss: 216283.2050061226 speed (wps):762.423939461976
step: 1198      loss: 242481.0287284851 speed (wps):762.1537785624325
Saving model parameters to best_params.pt
epoch: 1        train ppl: 319.86563318348607   val ppl: 275.072699700709       best val: 275.072699700709      time (s) spent in epoch: 1238.5455334186554

EPOCH 2 ------------------
step: 10        loss: 2237.92653799057  speed (wps):747.3242029639088
step: 142       loss: 28053.52163553238 speed (wps):761.9876604156635
step: 274       loss: 54356.591074466705        speed (wps):783.9244763034252
step: 406       loss: 80274.85654592514 speed (wps):812.5543017185942
step: 538       loss: 106365.77887296677        speed (wps):833.2770952132511
step: 670       loss: 132493.28870296478        speed (wps):845.4194815417692
step: 802       loss: 158134.52977657318        speed (wps):854.5973764630872
step: 934       loss: 183903.90338897705        speed (wps):846.3900259996085
step: 1066      loss: 209641.54880523682        speed (wps):832.095814460482
step: 1198      loss: 235115.7008910179 speed (wps):835.5292304773482
Saving model parameters to best_params.pt
epoch: 2        train ppl: 269.1902955017364    val ppl: 246.02598505865174     best val: 246.02598505865174    time (s) spent in epoch: 1120.8020842075348

EPOCH 3 ------------------
step: 10        loss: 2188.8030004501343        speed (wps):899.2219992800909
step: 142       loss: 27348.993027210236        speed (wps):902.2241227184151
step: 274       loss: 53097.99941778183 speed (wps):901.8645556226538
step: 406       loss: 78409.74533319473 speed (wps):901.0715822155596
step: 538       loss: 103925.39725065231        speed (wps):900.5218034094969
step: 670       loss: 129458.16264390945        speed (wps):900.3086486357212
step: 802       loss: 154512.99488544464        speed (wps):899.6741374322879
step: 934       loss: 179771.21260643005        speed (wps):898.1826573111662
step: 1066      loss: 204979.45505142212        speed (wps):897.9838274898742
step: 1198      loss: 229894.02508735657        speed (wps):897.9317948499602
Saving model parameters to best_params.pt
epoch: 3        train ppl: 238.04715005435955   val ppl: 230.03872458201718     best val: 230.03872458201718    time (s) spent in epoch: 1051.5604801177979

EPOCH 4 ------------------
step: 10        loss: 2145.196261405945 speed (wps):893.750869470297
step: 142       loss: 26805.494389533997        speed (wps):893.1453290268416
step: 274       loss: 52106.375386714935        speed (wps):893.9751193702921
step: 406       loss: 76931.94136619568 speed (wps):894.7747020426268
step: 538       loss: 101979.91278648376        speed (wps):895.2599264182646
step: 670       loss: 127064.57997322083        speed (wps):895.680142633723
step: 802       loss: 151649.32209968567        speed (wps):896.3083975289065
step: 934       loss: 176460.16721487045        speed (wps):896.1156240012864
step: 1066      loss: 201272.98335313797        speed (wps):896.1272251852181
step: 1198      loss: 225750.70413827896        speed (wps):896.4401033426635
Saving model parameters to best_params.pt
epoch: 4        train ppl: 215.87277186569682   val ppl: 221.251970920005       best val: 221.251970920005      time (s) spent in epoch: 1052.147334098816

# Problem 2

In [ ]:
# Problem 2
class GRU(nn.Module): # Implement a stacked GRU RNN
  """
  Follow the same instructions as for RNN (above), but use the equations for 
  GRU, not Vanilla RNN.
  """
  def __init__(self, emb_size, hidden_size, seq_len, batch_size, vocab_size, num_layers, dp_keep_prob):
    super(GRU, self).__init__()

    # TODO ========================

  def init_weights_uniform(self):
    # TODO ========================

  def init_hidden(self):
    # TODO ========================
    return # a parameter tensor of shape (self.num_layers, self.batch_size, self.hidden_size)

  def forward(self, inputs, hidden):
    # TODO ========================
    return logits.view(self.seq_len, self.batch_size, self.vocab_size), hidden

  def generate(self, input, hidden, generated_seq_len):
    # TODO ========================
    return samples

# Problem 3

In [ ]:
# Problem 3
##############################################################################
#
# Code for the Transformer model
#
##############################################################################

"""
Implement the MultiHeadedAttention module of the transformer architecture.
All other necessary modules have already been implemented for you.

We're building a transfomer architecture for next-step prediction tasks, and 
applying it to sequential language modelling. We use a binary "mask" to specify 
which time-steps the model can use for the current prediction.
This ensures that the model only attends to previous time-steps.

The model first encodes inputs using the concatenation of a learned WordEmbedding 
and a (in our case, hard-coded) PositionalEncoding.
The word embedding maps a word's one-hot encoding into a dense real vector.
The positional encoding 'tags' each element of an input sequence with a code that 
identifies it's position (i.e. time-step).

These encodings of the inputs are then transformed repeatedly using multiple
copies of a TransformerBlock.
This block consists of an application of MultiHeadedAttention, followed by a 
standard MLP; the MLP applies *the same* mapping at every position.
Both the attention and the MLP are applied with Resnet-style skip connections, 
and layer normalization.

The complete model consists of the embeddings, the stacked transformer blocks, 
and a linear layer followed by a softmax.
"""

#This code has been modified from an open-source project, by David Krueger.
#The original license is included below:
#MIT License
#
#Copyright (c) 2018 Alexander Rush
#
#Permission is hereby granted, free of charge, to any person obtaining a copy
#of this software and associated documentation files (the "Software"), to deal
#in the Software without restriction, including without limitation the rights
#to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
#copies of the Software, and to permit persons to whom the Software is
#furnished to do so, subject to the following conditions:
#
#The above copyright notice and this permission notice shall be included in all
#copies or substantial portions of the Software.
#
#THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
#IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
#FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
#AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
#LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
#OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
#SOFTWARE.



#----------------------------------------------------------------------------------

# TODO: implement this class
class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, n_units, dropout=0.1):
        """
        n_heads: the number of attention heads
        n_units: the number of output units
        dropout: probability of DROPPING units
        """
        super(MultiHeadedAttention, self).__init__()
        # This sets the size of the keys, values, and queries (self.d_k) to all 
        # be equal to the number of output units divided by the number of heads.
        self.d_k = n_units // n_heads
        # This requires the number of n_heads to evenly divide n_units.
        assert n_units % n_heads == 0
        self.n_units = n_units 

        # TODO: create/initialize any necessary parameters or layers
        # Note: the only Pytorch modules you are allowed to use are nn.Linear 
        # and nn.Dropout
        
    def forward(self, query, key, value, mask=None):
        # TODO: implement the masked multi-head attention.
        # query, key, and value all have size: (batch_size, seq_len, self.n_units, self.d_k)
        # mask has size: (batch_size, seq_len, seq_len)
        # As described in the .tex, apply input masking to the softmax 
        # generating the "attention values" (i.e. A_i in the .tex)
        # Also apply dropout to the attention values.

        return # size: (batch_size, seq_len, self.n_units)






#----------------------------------------------------------------------------------
# The encodings of elements of the input sequence

class WordEmbedding(nn.Module):
    def __init__(self, n_units, vocab):
        super(WordEmbedding, self).__init__()
        self.lut = nn.Embedding(vocab, n_units)
        self.n_units = n_units

    def forward(self, x):
        #print (x)
        return self.lut(x) * math.sqrt(self.n_units)


class PositionalEncoding(nn.Module):
    def __init__(self, n_units, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, n_units)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, n_units, 2).float() *
                             -(math.log(10000.0) / n_units))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)



#----------------------------------------------------------------------------------
# The TransformerBlock and the full Transformer


class TransformerBlock(nn.Module):
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(TransformerBlock, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(ResidualSkipConnectionWithLayerNorm(size, dropout), 2)
 
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask)) # apply the self-attention
        return self.sublayer[1](x, self.feed_forward) # apply the position-wise MLP


class TransformerStack(nn.Module):
    """
    This will be called on the TransformerBlock (above) to create a stack.
    """
    def __init__(self, layer, n_blocks): # layer will be TransformerBlock (below)
        super(TransformerStack, self).__init__()
        self.layers = clones(layer, n_blocks)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class FullTransformer(nn.Module):
    def __init__(self, transformer_stack, embedding, n_units, vocab_size):
        super(FullTransformer, self).__init__()
        self.transformer_stack = transformer_stack
        self.embedding = embedding
        self.output_layer = nn.Linear(n_units, vocab_size)
        
    def forward(self, input_sequence, mask):
        embeddings = self.embedding(input_sequence)
        return F.log_softmax(self.output_layer(self.transformer_stack(embeddings, mask)), dim=-1)


def make_model(vocab_size, n_blocks=6, 
               n_units=512, n_heads=16, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(n_heads, n_units)
    ff = MLP(n_units, dropout)
    position = PositionalEncoding(n_units, dropout)
    model = FullTransformer(
        transformer_stack=TransformerStack(TransformerBlock(n_units, c(attn), c(ff), dropout), n_blocks),
        embedding=nn.Sequential(WordEmbedding(n_units, vocab_size), c(position)),
        n_units=n_units,
        vocab_size=vocab_size
        )
    
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model


#----------------------------------------------------------------------------------
# Data processing

def subsequent_mask(size):
    """ helper function for creating the masks. """
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, x, pad=0):
        self.data = x
        self.mask = self.make_mask(self.data, pad)
    
    @staticmethod
    def make_mask(data, pad):
        "Create a mask to hide future words."
        mask = (data != pad).unsqueeze(-2)
        mask = mask & Variable(
            subsequent_mask(data.size(-1)).type_as(mask.data))
        return mask


#----------------------------------------------------------------------------------
# Some standard modules

class LayerNorm(nn.Module):
    "layer normalization, as in: https://arxiv.org/abs/1607.06450"
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


class ResidualSkipConnectionWithLayerNorm(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(ResidualSkipConnectionWithLayerNorm, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))


class MLP(nn.Module):
    """
    This is just an MLP with 1 hidden layer
    """
    def __init__(self, n_units, dropout=0.1):
        super(MLP, self).__init__()
        self.w_1 = nn.Linear(n_units, 2048)
        self.w_2 = nn.Linear(2048, n_units)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))